In [ ]:
import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

Annotations.

In [ ]:
import json
import pandas as pd

sats = []

with open('data/spotGEO/train_anno.json') as f:
    for ann in json.load(f):
        for i, coords in enumerate(ann['object_coords']):
            sats.append({
                'sequence': ann['sequence_id'],
                'frame': ann['frame'],
                'satellite': i + 1,
                'x': coords[0],
                'y': coords[1],
            })
    
sats = pd.DataFrame(sats)
sats = sats.set_index(['sequence', 'frame', 'satellite'])
sats.head()

Train images.

In [ ]:
import pathlib
from PIL import Image
import numpy as np

train = {
    (int(seq.name), int(frame.stem)): np.asarray(Image.open(frame)).astype(np.float32)
    for seq in pathlib.Path('data/spotGEO/train').glob('*')
    for frame in seq.glob('*.png')
}

Sequence visualisation.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from mpl_toolkits.axes_grid1 import ImageGrid

def viz_sequence(seq, df):
    
    # Make one image for each frame
    fig = plt.figure(figsize=(15, 10 * 5)) # 5 frames in each sequence
    grid = ImageGrid(
        fig, 111,  
        nrows_ncols=(5, 1),
        axes_pad=.4,
    )

    frames = df.loc[seq]
    trajectories = []

    for frame, sats in frames.groupby('frame'):
    
        # Sky
        ax = grid[frame - 1]
        ax.imshow(train[seq, frame], origin='lower')
        ax.set_title(f'Sequence #{seq}, frame #{frame}')

        # Current positions
        for _, sat in sats.iterrows():
            ax.scatter(sat['x'], sat['y'], s=100, facecolors='none', edgecolors='red', linewidths=2)
            
        # Old positions
        for old_sats in trajectories:
            for _, sat in old_sats.iterrows():
                ax.scatter(sat['x'], sat['y'], s=100, facecolors='none', edgecolors='yellow', linewidths=2)
        trajectories.append(sats)
        
        ax.set_xlim(-.5, 639.5)
        ax.set_ylim(479.5, -.5)
    
    return fig

fig = viz_sequence(19, df=sats)